# Simulating the Deutsch–Jozsa algorithm with noise

## Abstract

In this sample, we'll look at how noise in quantum devices may affect quantum algorithms such as the _Deutsch–Jozsa algorithm_, using the open systems simulator provided with the Quantum Development Kit to test Q# applications against different kinds of noise.

> **💡 TIP:** To learn more about the Deutsch–Jozsa algorithm check out the [**getting-started/simple-algorithms** sample](../../getting-started/simple-algorithms/README.md).

## Preamble

We start by importing the interoperability package that allows for calling into Q# from Python. This package allows us to define Q# functions and operations from within Python notebooks, and to simulate running quantum programs on various different simulators. This can take 15 seconds to run the first time, as the Q# environment is initializing.

In [1]:
import qsharp

## Deutsch–Jozsa without noise

Before seeing how the Deutsch–Jozsa algorithm works in the presence of noise, it helps to first test that our implementation is correct when run on an ideal (noiseless) quantum device. To do so, we'll start by writing an implementation of the Deutsch–Jozsa algorithm using the `%%qsharp` magic command. This can take 15 seconds to run the first time, as the Q# environment is initializing.

In [2]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

The main part of our implementation will be an operation that applies an oracle to a register in the $\left|++\cdots+-\right\rangle$ state, such that phase kickback from the last qubit can be observed on the control register.

Suppose that the oracle can be simulated by a unitary operator $U_f \left|x\right\rangle \left|y\right\rangle = \left|x\right\rangle \left|y \oplus f(x) \right\rangle$. Then, if $f(x)$ is a constant function such that $f(x) = s$ for all $x$, we have that
$$
\begin{aligned}
    U_f \left|++\cdots+-\right\rangle & = \frac{1}{\sqrt{2^n}} \left[
        (-1)^s \left|00\cdots0\right\rangle +
        (-1)^s \left|00\cdots1\right\rangle +
        (-1)^s \left|01\cdots0\right\rangle +
        (-1)^s \left|01\cdots1\right\rangle + \cdots
    \right] \otimes \left|-\right\rangle \\
    & = (-1)^s \left|++\cdots+-\right\rangle,
\end{aligned}
$$
where $n$ is the size of the register that our oracle acts on. Since $(-1)^s$ is a global phase, it has no effect on any measurement outcomes, such that we can measure the control register to see if it was left in the $\left|++\cdots+\right\rangle$ state by applying our oracle. If so, then we can conclude that $f$ must have been constant.

In [3]:
%%qsharp
operation MeasureIfConstantBooleanFunction(oracle : ((Qubit[], Qubit) => Unit), n : Int) : Bool {
    use queryRegister = Qubit[n];
    use target = Qubit();

    // The last qubit needs to be flipped so that the function will
    // actually be computed into the phase when Uf is applied.
    X(target);
    H(target);

    within {
        ApplyToEachA(H, queryRegister);
    } apply {
        // We now apply Uf to the n + 1 qubits, computing |𝑥, 𝑦〉 ↦ |𝑥, 𝑦 ⊕ 𝑓(𝑥)〉.
        oracle(queryRegister, target);
    }

    let resultArray = ForEach(MResetZ, queryRegister);
    Reset(target);

    return All(IsResultZero, resultArray);
}

Next, we need a Q# operation to represent the oracle itself. To do so, we'll consider an oracle representing the function
$$
    f(x) = \begin{cases}
        1 & x \in S \\
        0 & \text{otherwise}
    \end{cases}
$$
for some set of marked inputs $S$. That is, where $f$ is specified by a list of values for which $f(x) = 1$. Using this representation, we can apply our oracle to a given register using the [`ControlledOnInt` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint).

In [4]:
%%qsharp
operation ApplyOracle(n : Int, markedElements : Int[], query : Qubit[], target : Qubit) : Unit {
    for markedElement in markedElements {
        // Note: As X accepts a Qubit, and ControlledOnInt only
        // accepts Qubit[], we use ApplyToEachCA(X, _) which accepts
        // Qubit[] even though the target is only 1 Qubit.
        ControlledOnInt(markedElement, ApplyToEachCA(X, _))(query, [target]);
    }
}

Finally, we can use the two operations we defined above to run the Deutsch–Jozsa algorithm for an oracle given by a list of marked elements:

In [5]:
# Not required, but can be helpful to provide type information
# to some IDEs.
RunDeutschJozsa: qsharp.QSharpCallable = None

In [6]:
%%qsharp
operation RunDeutschJozsa(nQubits : Int, markedElements : Int[]) : Bool {
    return MeasureIfConstantBooleanFunction(
        ApplyOracle(nQubits, markedElements, _, _),
        nQubits
    );
}

For two qubits, marking `[]` and `[0, 1, 2, 3]` represent constant functions while lists of marked inputs such as `[0, 1]` `[2, 3]` represent balanced functions.

In [7]:
RunDeutschJozsa(nQubits=2, markedElements=[])

True

In [8]:
RunDeutschJozsa(nQubits=2, markedElements=[0, 1, 2, 3])

True

In [9]:
RunDeutschJozsa(nQubits=2, markedElements=[0, 1])

False

## Simulating with noise

To see how the Deutsch–Jozsa algorithm performs in the presence of noise, we can use the open systems simulator provided with the Quantum Development Kit.

In [10]:
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

The open systems simulator can be configured to use different kinds of noise by using the [QuTiP library](https://qutip.org/) to describe noise models for quantum devices, so we'll go on and import that library here.

In [11]:
import qutip as qt

For example, suppose that when the `H` operation is used, there's a small chance that it instead resets its input qubit to the $\left|0\right\rangle$ state instead. We can model this as _amplitude damping noise_, using QuTiP to write out the relevant operator.

> **💡 TIP:** To learn more about representing noise, check out the [open systems concept guide](https://github.com/microsoft/qsharp-runtime/blob/main/documentation/examples/open-systems-concepts.ipynb) provided with the Q# runtime.
> To learn more about amplitude damping noise in particular, check out [the summary on Wikipedia](https://en.wikipedia.org/wiki/Amplitude_damping_channel).

In particular, amplitude damping models a finite probability (represented below by `strength`) that the state of a qubit will be lost and replaced by $\left|0\right\rangle$.

In [12]:
def amplitude_damping_channel(strength):
    return qt.Qobj([
        [1.0, 0.0, 0.0, strength],
        [0.0, 1 - (strength / 2), 0.0, 0.0],
        [0.0, 0.0, 1 - (strength / 2), 0.0],
        [0.0, 0.0, 0.0, 1 - strength]
    ], dims=[[[2], [2]], [[2], [2]]])

We can then set the noise model for our open systems simulator to use amplitude damping noise to describe what happens when we call `H`:

In [13]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal')
noise_model['h'] = amplitude_damping_channel(strength=0.05) * qt.to_super(qt.qip.operations.hadamard_transform())
qsharp.experimental.set_noise_model(noise_model)

Whereas when running without noise, the Deutsch–Jozsa algorithm finds the right answer 100% of the time, noise causes our implementation to sometimes return the wrong answer; running 100 times, we thus won't see 100% accuracy in this case.

In [14]:
sum(RunDeutschJozsa.simulate_noise(nQubits=2, markedElements=[0, 1, 2, 3]) for _ in range(100))

89

Of course, we use more than just the `H` operation in our Deutsch–Jozsa algorithm, so let's go on and add some noise to other operations as well. For example, `X` may introduce extra bit flip errors while `Z` may introduce extra phase flip errors:

In [15]:
def bit_flip_channel(strength=0.05):
    return (1 - strength) * qt.to_super(qt.qeye(2)) + strength * qt.to_super(qt.sigmax())
def phase_flip_channel(strength=0.05):
    return (1 - strength) * qt.to_super(qt.qeye(2)) + strength * qt.to_super(qt.sigmaz())

In [16]:
noise_model['x'] = bit_flip_channel() * qt.to_super(qt.sigmax())
noise_model['z'] = phase_flip_channel() * qt.to_super(qt.sigmaz())
qsharp.experimental.set_noise_model(noise_model)

In [17]:
sum(RunDeutschJozsa.simulate_noise(nQubits=2, markedElements=[0, 1, 2, 3]) for _ in range(100))

76

## Further resources

To learn more about the Deutsch–Jozsa algorithm, check out:

- The [section on oracles](https://docs.microsoft.com/azure/quantum/concepts-oracles) in the Quantum Development Kit documentation.
- Chapter 8 of [_Learn Quantum Computing with Python and Q#_](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp).
- The [Deutsch–Jozsa kata](https://github.com/microsoft/QuantumKatas/tree/main/DeutschJozsaAlgorithm).

To learn more about the open systems simulator, check out:

- The [**characterization** samples](../../characterization/).
- The [section on noisy simulation](https://docs.microsoft.com/en-us/azure/quantum/user-guide/machines/noise-simulator) in the Quantum Development Kit documentation.
- The [**open-systems-concepts** guide](https://github.com/microsoft/qsharp-runtime/blob/main/documentation/examples/open-systems-concepts.ipynb) provided with the Q# runtime.

## Epilogue

In [18]:
qsharp.component_versions()

{'iqsharp': LooseVersion ('0.19.2109165653'),
 'Jupyter Core': LooseVersion ('1.5.0.0'),
 '.NET Runtime': LooseVersion ('.NETCoreApp,Version=v3.1'),
 'qsharp': LooseVersion ('0.19.2109.165653'),
 'experimental': {'simulators': {'features': ['DEFAULT'],
   'name': 'Microsoft.Quantum.Experimental.Simulators',
   'opt_level': '3',
   'target': 'x86_64-pc-windows-msvc',
   'version': '0.19.2109165653'}}}